<a href="https://colab.research.google.com/github/namans-git/res-paper-du/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [17]:
from google.colab import files
uploaded = files.upload()

Saving Front.xlsx to Front (1).xlsx


svm is a supervised learning model with associated learning algorithms that analyze data and "recognize patterns", generally used for "classification" and regression analysis.

what we'll be doing with svm is we'll assign the suspect students found using dissimilarity matrices into a category and all the other students in another and then svm will assign new examples into one category or other.

In [18]:
df = pd.read_excel('Front.xlsx')

In [19]:
#converting the submission time strings to datetime and then to epoch

df['Attempt Submission Time'] = pd.to_datetime(df['Attempt Submission Time'])
df['Attempt Submission Time']= df['Attempt Submission Time'].apply(lambda x: x.value)

In [20]:
#scaling numerics

scaler = MinMaxScaler()
df['std_scaled_grades'] = scaler.fit_transform(df['Grades'].values.reshape(-1,1))
df['std_scaled_duration'] = scaler.fit_transform(df['Exam duration (Sec)'].values.reshape(-1,1))
df['std_scaled_submTime'] = scaler.fit_transform(df['Attempt Submission Time'].values.reshape(-1,1))

In [21]:
# assigning every student default behaviour i.e. 0
df['behaviour'] = 0

# to these students, we assign 1
lst = [16, 18, 22, 24, 30, 34, 35, 36, 37, 38]
for k in lst:
  df['behaviour'][k] = 1

<ipython-input-21-942cbd9603d1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['behaviour'][k] = 1


In [22]:
df.loc[df['behaviour']==1]#the suspect students

,S.No.,Student,Attempt Submission Time,Grades,Exam duration (Sec),std_scaled_grades,std_scaled_duration,std_scaled_submTime,behaviour
16,17,Aditya P,1640701920000000000,73.33,1047,0.499906,0.674253,0.248792,1
18,19,SUJEET KUMAR,1640701260000000000,73.33,1027,0.499906,0.660028,0.242150,1
22,23,RAVI GUPTA,1640699880000000000,93.33,600,0.874930,0.356330,0.228261,1
24,25,P SANDEEP,1640698500000000000,93.33,581,0.874930,0.342817,0.214372,1
30,31,UTSAV SHARMA,1640697720000000000,93.33,596,0.874930,0.353485,0.206522,1
34,35,DIVYA GOEL,1640696640000000000,80.00,746,0.624977,0.460171,0.195652,1
35,36,SANDHYA RAGHAV,1640696640000000000,80.00,760,0.624977,0.470128,0.195652,1
36,37,SUDHIR NA,1640696640000000000,80.00,746,0.624977,0.460171,0.195652,1
37,38,DISHA GUPTA,1640694840000000000,100.00,941,1.000000,0.598862,0.177536,1
38,39,ADITI SUKHIJA,1640694840000000000,100.00,960,1.000000,0.612376,0.177536,1


In [23]:
#creating a seperate df consisting of all  the scaled numerics
data = [df['std_scaled_duration'], df['std_scaled_grades'], df['std_scaled_submTime']]
headers = ['std_scaled_duration', 'std_scaled_grades', 'std_scaled_submTime']

new_df = pd.concat(data, axis=1, keys=headers)

In [24]:
new_df.head(2)

,std_scaled_duration,std_scaled_grades,std_scaled_submTime
0,0.206970,0.874930,1.000000
1,0.445946,0.624977,0.923309


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X = new_df #X is going to be my feature dataframe
y = df['behaviour'] #target var

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [27]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

In [29]:
print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test,predictions))

[[10  0]
 [ 4  0]]


              precision    recall  f1-score   support

           0       0.71      1.00      0.83        10
           1       0.00      0.00      0.00         4

    accuracy                           0.71        14
   macro avg       0.36      0.50      0.42        14
weighted avg       0.51      0.71      0.60        14



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tuning the hyper-parameters of an estimator.

**Grid search** is commonly used as an approach to hyper-parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid.

In [30]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.1,1,10,100,1000], 'gamma': [1,0.1,0.01,0.001,0.0001]}
grid = GridSearchCV(SVC(),param_grid, verbose=3)

In [31]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ....................C=0.1, gamma=1;, score=0.714 total time=   0.0s
[CV 2/5] END ....................C=0.1, gamma=1;, score=0.833 total time=   0.0s
[CV 3/5] END ....................C=0.1, gamma=1;, score=0.833 total time=   0.0s
[CV 4/5] END ....................C=0.1, gamma=1;, score=0.833 total time=   0.0s
[CV 5/5] END ....................C=0.1, gamma=1;, score=0.833 total time=   0.0s
[CV 1/5] END ..................C=0.1, gamma=0.1;, score=0.714 total time=   0.0s
[CV 2/5] END ..................C=0.1, gamma=0.1;, score=0.833 total time=   0.0s
[CV 3/5] END ..................C=0.1, gamma=0.1;, score=0.833 total time=   0.0s
[CV 4/5] END ..................C=0.1, gamma=0.1;, score=0.833 total time=   0.0s
[CV 5/5] END ..................C=0.1, gamma=0.1;, score=0.833 total time=   0.0s
[CV 1/5] END .................C=0.1, gamma=0.01;, score=0.714 total time=   0.0s
[CV 2/5] END .................C=0.1, gamma=0.01

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             verbose=3)

In [32]:
print(grid.best_params_)
print(grid.best_estimator_)

{'C': 100, 'gamma': 1}
SVC(C=100, gamma=1)


In [33]:
grid_predictions = grid.predict(X_test)

In [34]:
print(confusion_matrix(y_test, grid_predictions))
print('\n')
print(classification_report(y_test, grid_predictions))

[[10  0]
 [ 3  1]]


              precision    recall  f1-score   support

           0       0.77      1.00      0.87        10
           1       1.00      0.25      0.40         4

    accuracy                           0.79        14
   macro avg       0.88      0.62      0.63        14
weighted avg       0.84      0.79      0.74        14

